In [1]:
import re
import pandas as pd
import numpy as np

##Extrinsic Eval

In [9]:
import pandas as pd
import numpy as np
import re
import nltk
import contractions

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import spacy
nlp = spacy.load("en_core_web_sm")

from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer,TreebankWordTokenizer
from nltk.tokenize import wordpunct_tokenize,TweetTokenizer



stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))



from autocorrect import Speller
spell = Speller(lang='en')
def autospell(text):
    spells = [spell(w) for w in (nltk.word_tokenize(text))]
    return " ".join(spells)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
def pos_tagger(tag):
    nltk_tag=tag[1]
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV   
    return 'n'

In [11]:
stop_list=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves',\
     'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', \
     'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',\
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'if', 'or', 'because', 'as', 'until','of', 'at', 'by', 'for', 'with', 'about', 'between',\
    'into', 'through', 'during', 'to', 'from','in', 'out','on','again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 
    'just','should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y','u','im','go','will','come','whats','twitter','tweet',\
    'know','x','yeah','year','yet','youre','would','do','can','nan','see','look','one','could']

stop_reg = r'\b(?:'+'|'.join(stop_list)+r')\b'
stop_reg
stopword_finder= re.compile(stop_reg)

In [14]:
def processing(sentence, verbose=False):
  og_sentence = sentence
  sentence=sentence.lower()#CASE FOLDING
  html_finder = re.compile(r'<.*?>|&\w+;')
  rmhtml_Text=html_finder.sub('',sentence)  #remove HTML Tags
  # rmurl_Text = re.sub(r'http\S+|https:\S+|www\S+|.com\S+', '', rmhtml_Text) #remove URL tags
  pre_url = r'(?:https?://)|(?:www\.)'
  url_set = r'[\w+=?/._\-:;&*^$#@~`!|]'
  rmurl_Text=re.sub(f'(?:{pre_url}){url_set}+|{url_set}+\.com{url_set}*', '',rmhtml_Text)
  username_finder = re.compile('@\w+')
  rmurl_Text=username_finder.sub('',rmurl_Text) #remove usernames
  rmwhite_Text=re.sub(' +', ' ',rmurl_Text) #remove extra whitespace
  rmwhite_Text = contractions.fix(rmwhite_Text) #Remove contractions
  rmpunt_text=re.sub('[^\w\s]','',rmwhite_Text) #punctuation and username removal
  
  token_text=[token for token in TweetTokenizer().tokenize(rmpunt_text)]#tokenize text 
  
  # stem_text= [stemmer.stem(word) for word in rmpunt_text]#stemming
  pos_tags=[pos_tagger(tag) for tag in nltk.pos_tag(token_text)]
  lem_text= [lemmatizer.lemmatize(word,pos=pos) for word,pos in zip(token_text,pos_tags)] #lemmatize word
  # print(lem_text)
  rmstop_text = [tok for tok in lem_text if not stopword_finder.search(tok)]
  # print(rmstop_text)
  # if len(rmstop_text)==0:
  #   rmstop_text=['<s>','</s>']
  # if rmstop_text[0]!='<s>':
  #   rmstop_text.insert('<s>',0)
  # if rmstop_text[-1]!='</s>':
  #   rmstop_text.append('</s>')
  
  sentence=TreebankWordDetokenizer().detokenize(rmstop_text) #convert to sentence 
  spell_sentence = autospell(sentence)
  if verbose:
    print(f'Original sentence :\n{og_sentence}\n')
    print(f'Removing HTML and (Lowercasing):\n{rmhtml_Text}\n')
    print(f'Removing URL tags (and usernames):\n{rmurl_Text}\n')
    print(f'Removing whitespace(and expand contractions):\n{rmwhite_Text}\n')
    print(f'Remove punctuations:\n{rmpunt_text}\n')   
    print(f'Tokenize Text:\n{token_text}\n')
    print(f'Lemmatize Text:\n{lem_text}\n')
    print(f'Removing stop words Text:\n{rmstop_text}\n')
    print(f'Spelling Correction:\n{spell_sentence}\n')
  
  return spell_sentence

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
def train_and_evaluate(train_sentences, train_labels,
test_sentences, test_labels):
  '''
  parameters:
  train_sentences : list of training sentences
  train_labels : list of training labels
  test_sentences : list of test sentences
  test_labels : list of test labels
  output:
  accuracy : accuracy of the test set
  '''
  # Model building
  model = make_pipeline(TfidfVectorizer(), MultinomialNB())
  # Training the model with the training data
  model.fit(train_sentences, train_labels)
  # Predicting the test data categories
  predicted_test_labels = model.predict(test_sentences)
  return accuracy_score(test_labels, predicted_test_labels)

In [28]:

df=pd.read_csv('/content/drive/MyDrive/NLP Assignments/A1_dataset_processed.csv')
#train sentences to list
train_sent_A= df.NEW_TEXT.values.astype('U').tolist()
#train labels to list
train_labels_A=df['LABEL'].tolist()


In [34]:
df = pd.read_csv('combined_data_.csv')
#train sentences to list
train_sent_B2= df.TEXT.values.astype('U').tolist()
#train labels to list
train_labels_B2=df['LABEL'].tolist()


In [35]:
# df=pd.read_csv('/content/drive/MyDrive/NLP Assignments/A1_dataset_processed.csv') #modify this for dataset B
df = pd.read_csv('/content/drive/MyDrive/NLP Assignments/combined_data.csv')
#train sentences to list
train_sent_B1= df.TEXT.values.astype('U').tolist()
#train labels to list
train_labels_B1=df['LABEL'].tolist()


In [27]:
#read test csv
test_df=pd.read_csv('/content/drive/MyDrive/NLP Assignments/A2_test_dataset.csv')
#pre-process test data
data=test_df.apply(lambda row: processing(row.TEXT,verbose=False),axis=1)
test_df['NEW_TEXT']= data

In [29]:
#test sentences to list
test_sent=test_df['NEW_TEXT'].values.astype('U').tolist()
#test labels to list
test_labels=test_df['LABEL'].tolist()

In [30]:
print(f'The accuracy of Test set using dataset A for training is :{train_and_evaluate(train_sent_A,train_labels_A,test_sent,test_labels)}')

The accuracy of Test set using dataset A for training is :0.8509316770186336


In [37]:
print(f'The accuracy of Test set using dataset B for training is :{train_and_evaluate(train_sent_B2,train_labels_B2,test_sent,test_labels)}')

The accuracy of Test set using dataset B for training is :0.8804347826086957


In [36]:
print(f'The accuracy of Test set using dataset B for training is :{train_and_evaluate(train_sent_B1,train_labels_B1,test_sent,test_labels)}')

The accuracy of Test set using dataset B for training is :0.8913043478260869
